# Team information

|Team-number :| 1|
|:----:|:----:|


|Name|    E-Mail        |matriculation-nr.|
|:----:|:----:|:----:|
|Tamara Scherer| schere21@ads.uni-passau.de|104218|
|Felix Müller| muell518@ads.uni-passau.de|104227|


In [ ]:
pip install label_studio

In [ ]:
pip install label-studio-sdk

In [ ]:

from label_studio_sdk import Client
from label_studio_sdk import project
from label_studio_sdk import project
import pandas as pd
LABEL_STUDIO_URL = 'http://132.231.59.226:8080' #this address needs to be the same as the address the label-studio is hosted on.
API_KEY = '1655a8922f821195356a17a3224c0532b091c61d' #please add your personal API_Key here to get your API_Key follow the Pictures below

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()
pro = project.Project.get_from_id(ls,"1")
tasks = project.Project.get_labeled_tasks(pro)
tasks[0]

In [ ]:
import re

In [ ]:
#TBD
re.findall("Q" ,tasks)